In [67]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [68]:
with open('MANY_OUTLIERS.json') as file:
    data = json.load(file)


In [69]:
rbf_pts = pd.DataFrame(data['rbfpoints'], columns=['i', 'j', 'x', 'y'])
rbf_pts['name'] = 'rbfpoints'

In [70]:
fig = px.scatter(rbf_pts, x="x", y="y", hover_data={
    "i":':.0f', 
    "j":':.0f',
    "x":':.1f',
    "y":':.1f',
    }, color="name")

fig.update_layout(
    yaxis = dict(autorange="reversed")
)

fig.show()


In [71]:
rbf_pts

,i,j,x,y,name
0,-7,-3,311.564232,1440.797321,rbfpoints
1,-8,-4,231.325562,1359.975453,rbfpoints
2,6,-10,1353.785066,881.681599,rbfpoints
3,6,9,1353.779820,2401.682088,rbfpoints
4,-7,2,311.564101,1840.797293,rbfpoints
...,...,...,...,...,...
575,-7,12,311.569440,2640.808073,rbfpoints
576,10,-6,1649.280106,1197.479679,rbfpoints
577,-9,-5,151.058115,1278.616681,rbfpoints
578,-9,7,151.058155,2238.613709,rbfpoints


In [72]:
# Supponiamo che rbf_pts sia il DataFrame di input
# Aggiungiamo una colonna di indici espliciti
rbf_pts['index'] = rbf_pts.index

# Creiamo delle copie delle colonne 'i' e 'j' per facilitare le operazioni di merge successivo
rbf_pts['key_i'] = rbf_pts['i']
rbf_pts['key_j'] = rbf_pts['j']

# Adiacenti j (up e down)
j_up = rbf_pts.copy()
j_up['j'] = j_up['j'] - 1  # punti adiacenti in j verso l'alto
merged_up = rbf_pts.merge(j_up, on=['i', 'j'], suffixes=('', '_up'))

j_down = rbf_pts.copy()
j_down['j'] = j_down['j'] + 1  # punti adiacenti in j verso il basso
merged_down = rbf_pts.merge(j_down, on=['i', 'j'], suffixes=('', '_down'))

# Adiacenti i (left e right)
i_left = rbf_pts.copy()
i_left['i'] = i_left['i'] - 1  # punti adiacenti in i verso sinistra
merged_left = rbf_pts.merge(i_left, on=['i', 'j'], suffixes=('', '_left'))

i_right = rbf_pts.copy()
i_right['i'] = i_right['i'] + 1  # punti adiacenti in i verso destra
merged_right = rbf_pts.merge(i_right, on=['i', 'j'], suffixes=('', '_right'))

# Aggiungere le liste degli indici degli adiacenti
rbf_pts['adiacenti_j'] = rbf_pts.apply(lambda row: merged_up.loc[merged_up['index'] == row['index'], 'index_up'].tolist(
) + merged_down.loc[merged_down['index'] == row['index'], 'index_down'].tolist(), axis=1)
rbf_pts['adiacenti_i'] = rbf_pts.apply(lambda row: merged_left.loc[merged_left['index'] == row['index'], 'index_left'].tolist(
) + merged_right.loc[merged_right['index'] == row['index'], 'index_right'].tolist(), axis=1)

# Ora rimuoviamo le colonne temporanee
rbf_pts.drop(columns=['key_i', 'key_j'], inplace=True)

In [73]:
def calculate_distances(row):
    # Supponiamo che il primo elemento in adiacenti_j sia 'up' e il secondo sia 'down'
    if len(row['adiacenti_j']) > 0:
        adiacenti_j = row['adiacenti_j']
        if adiacenti_j[0] < len(rbf_pts):
            row['y_dist_up'] = rbf_pts.loc[adiacenti_j[0], 'y'] - row['y']
        if len(adiacenti_j) > 1 and adiacenti_j[1] < len(rbf_pts):
            row['y_dist_down'] = rbf_pts.loc[adiacenti_j[1], 'y'] - row['y']

    # Supponiamo che il primo elemento in adiacenti_i sia 'right' e il secondo sia 'left'
    if len(row['adiacenti_i']) > 0:
        adiacenti_i = row['adiacenti_i']
        if adiacenti_i[0] < len(rbf_pts):
            row['x_dist_right'] = rbf_pts.loc[adiacenti_i[0], 'x'] - row['x']
        if len(adiacenti_i) > 1 and adiacenti_i[1] < len(rbf_pts):
            row['x_dist_left'] = rbf_pts.loc[adiacenti_i[1], 'x'] - row['x']

    return row

rbf_pts = rbf_pts.apply(calculate_distances, axis=1)

In [74]:
rbf_pts

,adiacenti_i,adiacenti_j,i,index,j,name,x,x_dist_left,x_dist_right,y,y_dist_down,y_dist_up
0,"[332, 281]","[206, 279]",-7,0,-3,rbfpoints,311.564232,-80.238761,80.219522,1440.797321,-79.999587,80.000622
1,"[279, 530]","[281, 207]",-8,1,-4,rbfpoints,231.325562,-80.267210,80.238464,1359.975453,-79.999360,79.999400
2,"[208, 480]","[225, 430]",6,2,-10,rbfpoints,1353.785066,-80.151200,79.741195,881.681599,-79.999683,79.999972
3,"[454, 333]","[377, 265]",6,3,9,rbfpoints,1353.779820,-80.143051,79.754863,2401.682088,-80.000481,79.997887
4,"[151, 41]","[42, 112]",-7,4,2,rbfpoints,311.564101,-80.238559,80.219922,1840.797293,-80.000345,80.000858
...,...,...,...,...,...,...,...,...,...,...,...,...
575,"[487, 526]",[340],-7,575,12,rbfpoints,311.569440,-80.242604,80.208957,2640.808073,NaN,-80.070841
576,[446],"[510, 569]",10,576,-6,rbfpoints,1649.280106,NaN,-53.957433,1197.479679,-79.997964,79.998362
577,[207],"[530, 512]",-9,577,-5,rbfpoints,151.058115,NaN,80.267529,1278.616681,-79.997443,79.999879
578,[473],"[531, 513]",-9,578,7,rbfpoints,151.058155,NaN,80.267536,2238.613709,-79.998330,80.009247


In [83]:
# Supponiamo che rbf_pts sia il DataFrame con le colonne già calcolate
plot_data = []

# Iteriamo sui punti e aggiungiamo i loro adiacenti
for idx, row in rbf_pts.iterrows():
    # Aggiungi il punto principale
    plot_data.append({'x': row['x'], 'y': row['y'],
                     'i': row['i'], 'j': row['j'], 'group': f'Point {idx}',
                     'adiacenti_i': row['adiacenti_i'], 'adiacenti_j': row['adiacenti_j'],
                     'x_dist_left': row['x_dist_left'], 'x_dist_right': row['x_dist_right'],
                     'y_dist_down': row['y_dist_down'], 'y_dist_up': row['y_dist_up']})

    # Aggiungi i punti adiacenti
    adiacenti_i = row['adiacenti_i']
    adiacenti_j = row['adiacenti_j']

    for adj_idx in adiacenti_i + adiacenti_j:
        adj_point = rbf_pts.loc[adj_idx]
        plot_data.append({'x': adj_point['x'], 'y': adj_point['y'],
                         'i': adj_point['i'], 'j': adj_point['j'], 'group': f'Point {idx}',
                         'adiacenti_i': adj_point['adiacenti_i'], 'adiacenti_j': adj_point['adiacenti_j'],
                         'x_dist_left': adj_point['x_dist_left'], 'x_dist_right': adj_point['x_dist_right'],
                         'y_dist_down': adj_point['y_dist_down'], 'y_dist_up': adj_point['y_dist_up']})

# Convertiamo la lista in un DataFrame
plot_data_df = pd.DataFrame(plot_data)

# Creare il grafico in coordinate x, y
fig_xy = go.Figure()

# Aggiungi tutti i punti con colore nero come prima entry della legenda
fig_xy.add_trace(go.Scatter(
    x=rbf_pts['x'], y=rbf_pts['y'], mode='markers',
    marker=dict(color='black', size=5),
    name='All Points',
    hovertemplate=(
        'x: %{x}<br>' +
        'y: %{y}<br>' +
        'i: %{text[0]}<br>' +
        'j: %{text[1]}<br>' +
        'adiacenti_i: %{text[2]}<br>' +
        'adiacenti_j: %{text[3]}<br>' +
        'x_dist_left: %{text[4]}<br>' +
        'x_dist_right: %{text[5]}<br>' +
        'y_dist_down: %{text[6]}<br>' +
        'y_dist_up: %{text[7]}<br>' +
        '<extra></extra>'
    ),
    text=[plot_data_df.loc[i, ['i', 'j', 'adiacenti_i', 'adiacenti_j', 'x_dist_left', 'x_dist_right', 'y_dist_down', 'y_dist_up']].values.tolist() for i in range(len(plot_data_df))]
))

# Aggiungi ogni gruppo di punti con i loro adiacenti
for group in plot_data_df['group'].unique():
    group_data = plot_data_df[plot_data_df['group'] == group]
    fig_xy.add_trace(go.Scatter(x=group_data['x'], y=group_data['y'], mode='markers',
                                marker=dict(size=10), name=group))

# Configurazione del layout
fig_xy.update_layout(title="Punti e i loro adiacenti (coordinate x, y)",
                     xaxis_title="x", yaxis_title="y", yaxis=dict(autorange="reversed"))

# Creare il grafico in coordinate i, j
fig_ij = go.Figure()

# Aggiungi tutti i punti con colore nero come prima entry della legenda
fig_ij.add_trace(go.Scatter(
    x=rbf_pts['i'], y=rbf_pts['j'], mode='markers',
    marker=dict(color='black', size=5),
    name='All Points',
    hovertemplate=(
        'i: %{x}<br>' +
        'j: %{y}<br>' +
        'x: %{text[0]}<br>' +
        'y: %{text[1]}<br>' +
        'adiacenti_i: %{text[2]}<br>' +
        'adiacenti_j: %{text[3]}<br>' +
        'x_dist_left: %{text[4]}<br>' +
        'x_dist_right: %{text[5]}<br>' +
        'y_dist_down: %{text[6]}<br>' +
        'y_dist_up: %{text[7]}<br>' +
        '<extra></extra>'
    ),
    text=[plot_data_df.loc[i, ['x', 'y', 'adiacenti_i', 'adiacenti_j', 'x_dist_left', 'x_dist_right', 'y_dist_down', 'y_dist_up']].values.tolist() for i in range(len(plot_data_df))]
))

# Aggiungi ogni gruppo di punti con i loro adiacenti
for group in plot_data_df['group'].unique():
    group_data = plot_data_df[plot_data_df['group'] == group]
    fig_ij.add_trace(go.Scatter(x=group_data['i'], y=group_data['j'], mode='markers',
                                marker=dict(size=10), name=group))

# Configurazione del layout
fig_ij.update_layout(
    title="Punti e i loro adiacenti (coordinate i, j)", xaxis_title="i", yaxis_title="j")

# Mostra entrambi i grafici
fig_xy.show()
fig_ij.show()



In [81]:
import pandas as pd
import plotly.graph_objects as go

# Supponiamo che rbf_pts sia il DataFrame con le colonne già calcolate
plot_data = []

# Iteriamo sui punti e aggiungiamo i loro adiacenti
for idx, row in rbf_pts.iterrows():
    # Aggiungi il punto principale
    plot_data.append({'x': row['x'], 'y': row['y'],
                     'i': row['i'], 'j': row['j'], 'group': f'Point {idx}',
                     'adiacenti_i': row['adiacenti_i'], 'adiacenti_j': row['adiacenti_j'],
                     'x_dist_left': row['x_dist_left'], 'x_dist_right': row['x_dist_right'],
                     'y_dist_down': row['y_dist_down'], 'y_dist_up': row['y_dist_up']})

    # Aggiungi i punti adiacenti
    adiacenti_i = row['adiacenti_i']
    adiacenti_j = row['adiacenti_j']

    for adj_idx in adiacenti_i + adiacenti_j:
        adj_point = rbf_pts.loc[adj_idx]
        plot_data.append({'x': adj_point['x'], 'y': adj_point['y'],
                         'i': adj_point['i'], 'j': adj_point['j'], 'group': f'Point {idx}',
                         'adiacenti_i': adj_point['adiacenti_i'], 'adiacenti_j': adj_point['adiacenti_j'],
                         'x_dist_left': adj_point['x_dist_left'], 'x_dist_right': adj_point['x_dist_right'],
                         'y_dist_down': adj_point['y_dist_down'], 'y_dist_up': adj_point['y_dist_up']})

# Convertiamo la lista in un DataFrame
plot_data_df = pd.DataFrame(plot_data)

# Creare il grafico in coordinate x, y
fig_xy = go.Figure()

# Aggiungi tutti i punti con colore nero come prima entry della legenda
fig_xy.add_trace(go.Scatter(
    x=rbf_pts['x'], y=rbf_pts['y'], mode='markers',
    marker=dict(color='black', size=5),
    name='All Points',
    customdata=plot_data_df[['i', 'j', 'adiacenti_i', 'adiacenti_j',
                             'x_dist_left', 'x_dist_right', 'y_dist_down', 'y_dist_up']],
    hovertemplate='<br>'.join([
        'x: %{y}',
        'y: %{x}',
        'i: %{customdata[0]}',
        'j: %{customdata[1]}',
        'adiacenti_i: %{customdata[2]}',
        'adiacenti_j: %{customdata[3]}',
        'x_dist_left: %{customdata[4]}',
        'x_dist_right: %{customdata[5]}',
        'y_dist_down: %{customdata[6]}',
        'y_dist_up: %{customdata[7]}',
    ],
    )))
# Aggiungi ogni gruppo di punti con i loro adiacenti
for group in plot_data_df['group'].unique():
    group_data = plot_data_df[plot_data_df['group'] == group]
    fig_xy.add_trace(go.Scatter(x=group_data['x'], y=group_data['y'], mode='markers',
                                marker=dict(size=10), name=group))

# Configurazione del layout
fig_xy.update_layout(title="Punti e i loro adiacenti (coordinate x, y)",
                     xaxis_title="x", yaxis_title="y", yaxis=dict(autorange="reversed"))

# Mostra entrambi i grafici
fig_xy.show()

In [76]:
# Definisci i parametri degli istogrammi
bin_count = 60  # Numero di bin predefinito
range_min = -150  # Range minimo predefinito
range_max = 150  # Range massimo predefinito

# Crea istogrammi per ogni colonna
hist_x_dist_left = go.Histogram(
    x=rbf_pts['x_dist_left'],
    name='x_dist_left',
    opacity=0.75,
    nbinsx=bin_count,
    xaxis='x',
    autobinx=False,
    xbins=dict(start=range_min, end=range_max)
)

hist_x_dist_right = go.Histogram(
    x=rbf_pts['x_dist_right'],
    name='x_dist_right',
    opacity=0.75,
    nbinsx=bin_count,
    xaxis='x',
    autobinx=False,
    xbins=dict(start=range_min, end=range_max)
)

hist_y_dist_down = go.Histogram(
    x=rbf_pts['y_dist_down'],
    name='y_dist_down',
    opacity=0.75,
    nbinsx=bin_count,
    xaxis='x',
    autobinx=False,
    xbins=dict(start=range_min, end=range_max)
)

hist_y_dist_up = go.Histogram(
    x=rbf_pts['y_dist_up'],
    name='y_dist_up',
    opacity=0.75,
    nbinsx=bin_count,
    xaxis='x',
    autobinx=False,
    xbins=dict(start=range_min, end=range_max)
)

# Combina tutti gli istogrammi in un'unica figura
fig = go.Figure(data=[hist_x_dist_left, hist_x_dist_right, hist_y_dist_down, hist_y_dist_up])

# Aggiungi il layout per permettere l'interazione tramite la legenda e aggiungi il pulsante di scala
fig.update_layout(
    title_text='Istogrammi delle distanze x e y',
    barmode='overlay',
    xaxis_title_text='Valori delle distanze',
    yaxis_title_text='Conteggio',
    yaxis_type='linear',  # Imposta la scala iniziale su lineare
    legend_title_text='Distanze',
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [{'yaxis.type': 'linear'}],
                    'label': 'Scala Lineare',
                    'method': 'relayout'
                },
                {
                    'args': [{'yaxis.type': 'log'}],
                    'label': 'Scala Logaritmica',
                    'method': 'relayout'
                }
            ],
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top'
        }
    ]
)

# Mostra la figura
fig.show()